In [5]:
from enum import Enum
from preprocessing_methods import *
from apify_scrap import *
from datetime import datetime
import mlflow
import pickle
from model_functions import *
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)
C:\Users\michi\AppData\Roaming\Python\Python38\site-packages\visions\backends\shared\nan_handling.py:51: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def hasna(x: np.ndarray) -> bool:


In [6]:
class ImmoWeltUrls(Enum):
    BUY_FLATS_WUE_10km = "https://www.immowelt.de/liste/wuerzburg/wohnungen/kaufen?d=true&r=10&sd=DESC&sf=RELEVANCE&sp=1"
    # add price range to avoid "consulting"-offers without named price
    BUY_HOUSES_WUE_10km = "https://www.immowelt.de/liste/wuerzburg/haeuser/kaufen?d=true&pma=10000000&pmi=10&r=10&sd=DESC&sf=RELEVANCE&sp=1"
    RENT_FLATS_WUE_10km = "https://www.immowelt.de/liste/wuerzburg/wohnungen/mieten?d=true&r=10&sd=DESC&sf=RELEVANCE&sp=1"
    RENT_HOUSES_WUE_10km = "https://www.immowelt.de/liste/wuerzburg/haeuser/mieten?d=true&r=10&sd=DESC&sf=RELEVANCE&sp=1"


def getFeatureSetApp():
    return [
        "Object_price",
        "LivingSpace",
        "ZipCode",
        "Rooms",
        "altbau_(bis_1945)",
        "balkon",
        "barriefrei",
        "dachgeschoss",
        "einbaukueche",
        "neubau",
        "parkett",
        "stellplatz",
        "bad/wc_getrennt",
        "personenaufzug",
        "garten",
        "garage",
        "renoviert",
        "terrasse",
        "wanne",
        "zentralheizung",
        "abstellraum",
        "ferne",
        "fussbodenheizung",
        "gartennutzung",
        "kelleranteil",
    ]

# Dataseti  issues

In [ ]:
import pandas as pd

X_train = pd.read_excel("data/X_train.xlsx")
y_train = pd.read_excel("data/y_train.xlsx")
X_val = pd.read_excel("data/X_val.xlsx")
y_val = pd.read_excel("data/y_val.xlsx")
X_test = pd.read_excel("data/X_test.xlsx")
y_test = pd.read_excel("data/X_test.xlsx")

# save a test, train val set as excel
# drop the index column of X_train
X_train = X_train.drop("Unnamed: 0", axis=1)
X_val = X_val.drop("Unnamed: 0", axis=1)

train = pd.concat([X_train, y_train], axis=1)
val = pd.concat([X_val, y_val], axis=1)
test = pd.concat([X_test, y_test], axis=1)
print(train.shape)
train = train.drop("Unnamed: 0", axis=1)
val = val.drop("Unnamed: 0", axis=1)
test = test.drop("Unnamed: 0", axis=1)
print(train.shape)
print(train.columns)

# train.to_excel("data/train2.xlsx", index=False)
# val.to_excel("data/val.xlsx", index=False)
# test.to_excel("data/test.xlsx", index=False)

print(train.columns)

In [57]:
def evaluate_model(model, X_train_recent, y_train_recent, X_val, y_val, X_test, y_test):
    pred_train = model.predict(X_train_recent)
    preds = model.predict(X_val)
    pred_test = model.predict(X_test)

    mae_train = mean_absolute_error(y_train_recent, pred_train)
    mse_train = mean_squared_error(y_train_recent, pred_train)
    r2_train = r2_score(y_train_recent, pred_train)

    mae_test = mean_absolute_error(y_test, pred_test)
    mse_test = mean_squared_error(y_test, pred_test)
    r2_test = r2_score(y_test, pred_test)

    mae_val = mean_absolute_error(y_val, preds)
    mse_val = mean_squared_error(y_val, preds)
    r2_val = r2_score(y_val, preds)

    mlflow.log_metric("mae_test", mae_test)
    mlflow.log_metric("mse_test", mse_test)
    mlflow.log_metric("r2_test", r2_test)
    mlflow.log_metric("mae_train", mae_train)
    mlflow.log_metric("r2_train", mae_train)
    mlflow.log_metric("mse_train", mae_train)
    mlflow.log_metric("mae", mae_val)
    mlflow.log_metric("mse", mse_val)
    mlflow.log_metric("r2", r2_val)
    return (
        mae_val,
        mse_val,
        r2_val,
        mae_test,
        mse_test,
        r2_test,
        mae_train,
        mse_train,
        r2_train,
    )

# Pipeline


In [26]:
def decode_col_names(df):
    df.columns = [
        re.sub(r"\\u([0-9a-fA-F]{4})", lambda m: chr(int(m.group(1), 16)), col)
        for col in df.columns
    ]
    df.columns = [
        col.replace("ö", "oe").replace("ä", "ae").replace("ü", "ue").replace("ß", "ss")
        for col in df.columns
    ]
    return df

In [58]:
def trigger_retraining_with_added_data(
    url,
    feature_set,
    limit=3,
    model_list=["ridge", "rf", "elasticnet", "linear"],
):
    # print(url)
    # #retrain_data = get_dataset_items(url, limit)
    # print("Retraining data successfully scraped.")
    # # write_data_to_excel(retrain_data, "data/retrain_train_data.xlsx")
    # print("Retraining data successfully written to excel.")

    # new_df = pd.read_excel(r"data/retrain_train_data.xlsx")
    # # new_df = preprocess_data(new_df)
    # print("Done with raw preprocessing.")
    # new_df.to_excel("data/retrain_train_data_preprocessed.xlsx", index=False)

    ############################# FELIX HILFE BENÖTIGT :) ##################################

    X_val = pd.read_excel("data/X_val.xlsx")
    X_val = X_val.drop("Unnamed: 0", axis=1)
    y_val = pd.read_excel("data/y_val.xlsx")
    y_val = y_val.drop("Unnamed: 0", axis=1)
    X_test = pd.read_excel("data/X_test.xlsx")
    X_test = X_test.drop("Unnamed: 0", axis=1)
    y_test = pd.read_excel("data/y_test.xlsx")
    y_test = y_test.drop("Unnamed: 0", axis=1)

    new_df = pd.read_excel("data/retrain_train_data_preprocessed.xlsx")
    new_df = decode_col_names(new_df)

    for feature in feature_set:
        if feature not in new_df.columns:
            new_df[feature] = 0

    train_recent = pd.read_excel("data/train_recent.xlsx")
    print("old shape of train_recent", train_recent.shape)

    new_df = preprocess_data_for_model(new_df, feature_set)
    train_recent = pd.concat([train_recent, new_df], axis=0)
    train_recent = train_recent.drop_duplicates()
    print("new shape of train_recent", train_recent.shape)
    print("Retraining data successfully added to training data.")
    train_recent = train_recent.fillna(0)
    train_recent.to_excel("data/train_recent_add.xlsx", index=False)

    print("train_recent shape before dropping na", train_recent.shape)
    print("train_recent shape after dropping na", train_recent.shape)
    y_train_recent = train_recent["Object_price"]
    X_train_recent = train_recent.drop(["Object_price"], axis=1)

    now = datetime.now()
    print(
        "!!!--------------------------------------START RETRAINING----------------------------------------------!!!"
    )

    model = None
    mlflow.set_experiment(f"retraining_{now.strftime('%Y-%m-%d_%H-%M')}")

    results = pd.DataFrame()
    for model_name in model_list:
        if model_name == "xgb":
            mlflow.xgboost.autolog()
        else:
            mlflow.sklearn.autolog()

        with mlflow.start_run(run_name=f"{model_name}"):
            if model_name == "xgb":
                print("XGB------")
                print(f"train{X_train_recent.shape}")
                print(f"val:{X_val.shape}")
                print(f"y_train:{y_train_recent.shape}")
                print(f"y_val:{y_val.shape}")
                model = train_and_eval_xgb(X_train_recent, y_train_recent, X_val, y_val)
            elif model_name == "lasso":
                print("LASSO------")
                model = train_and_eval_lasso(
                    X_train_recent, y_train_recent, X_val, y_val
                )
            elif model_name == "ridge":
                print("RIDGE------")
                model = train_and_eval_ridge(
                    X_train_recent, y_train_recent, X_val, y_val
                )
            elif model_name == "rf":
                print("RF------")
                model = train_and_eval_rf(X_train_recent, y_train_recent, X_val, y_val)
            elif model_name == "elasticnet":
                print("ELASTICNET------")
                model = train_and_eval_elasticnet(
                    X_train_recent, y_train_recent, X_val, y_val
                )
            elif model_name == "linear":
                print("LINEAR------")
                model = train_and_eval_linear(
                    X_train_recent, y_train_recent, X_val, y_val
                )
            else:
                print("Model not found.")

            print(f"Training {model_name} model done...")
            print(f"----------EVALUATION AND LOGGING TO MLFLOW-------- {model_name}")

            (
                mae_val,
                mse_val,
                r2_val,
                mae_test,
                mse_test,
                r2_test,
                mae_train,
                mse_train,
                r2_train,
            ) = evaluate_model(
                model, X_train_recent, y_train_recent, X_val, y_val, X_test, y_test
            )

            results = results.append(
                {
                    "model": model_name,
                    "mae": mae_val,
                    "mse": mse_val,
                    "r2": r2_val,
                    "mae_test": mae_test,
                    "mse_test": mse_test,
                    "r2_test": r2_test,
                    "mae_train": mae_train,
                    "mse_train": mse_train,
                    "r2_train": r2_train,
                },
                ignore_index=True,
            )

            print(f"Model {model_name} saved locally as pickle file.")
            print("Done with train")
            mlflow.end_run()

    return results


trigger_retraining_with_added_data(
    url="https://www.immowelt.de/liste/wuerzburg/wohnungen/mieten?d=true&r=10&sd=DESC&sf=RELEVANCE&sp=1",
    feature_set=getFeatureSetApp(),
)

old shape of train_recent (108, 47)
Index(['Object_price', 'LivingSpace', 'Rooms', 'ConstructionYear', 'ZipCode',
       'EstateType', 'DistributionType', 'abstellraum', 'altbau_(bis_1945)',
       'balkon', 'barriefrei', 'dachgeschoss', 'dsl', 'dusche', 'einbaukueche',
       'elektro', 'erdgeschoss', 'etagenheizung', 'fenster', 'fern', 'ferne',
       'fliesen', 'frei', 'fussbodenheizung', 'gaestewc', 'garten',
       'gartennutzung', 'gas', 'haustiere_erlaubt', 'kelleranteil',
       'kunststoff', 'kunststofffenster', 'luftwp', 'neubau', 'offene_kueche',
       'parkett', 'pellet', 'personenaufzug', 'reinigung', 'renoviert',
       'rollstuhlgerecht', 'sat', 'speisekammer', 'stellplatz', 'terrasse',
       'tiefgarage', 'wanne', 'wg_geeignet', 'zentralheizung',
       'bad/wc_getrennt', 'garage'],
      dtype='object')
Used feature set for preprocessing:['Object_price', 'LivingSpace', 'ZipCode', 'Rooms', 'altbau_(bis_1945)', 'balkon', 'barriefrei', 'dachgeschoss', 'einbaukueche', 'n

2023/07/15 01:41:57 INFO mlflow.tracking.fluent: Experiment with name 'retraining_2023-07-15_01-41' does not exist. Creating a new experiment.
2023/07/15 01:42:04 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\mlflow\data\pandas_dataset.py:116: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for mor

RIDGE------


2023/07/15 01:42:08 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\mlflow\models\signature.py:144: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2023/07/15 01:42:28 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\michi\Anaconda3\envs\e

Training ridge model done...
-------------------------EVALUATION AND LOGGING TO MLFLOW------------------------- ridge


2023/07/15 01:42:30 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\mlflow\data\pandas_dataset.py:116: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."
2023/07/15 01:42:32 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\michi\Anaconda3\env

Model ridge saved locally as pickle file.
Done with train


2023/07/15 01:42:57 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\mlflow\data\pandas_dataset.py:116: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."


RF------


2023/07/15 01:43:01 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "c:\Users\michi\Anaconda3\envs\enterpriseai2\lib\site-packages\mlflow\models\signature.py:144: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."


In [ ]:
# print(f"Training {model_name} model done...")
# print(f"Evaluating and Logging {model_name} model to MLFlow...")
# pred_train = model.predict(X_train_recent)
# preds = model.predict(X_val)
# pred_test = model.predict(X_test)

# mae_train = mean_absolute_error(y_train_recent, pred_train)
# mse_train = mean_squared_error(y_train_recent, pred_train)
# r2_train = r2_score(y_train_recent, pred_train)

# mae_test = mean_absolute_error(y_test, pred_test)
# mse_test = mean_squared_error(y_test, pred_test)
# r2_test = r2_score(y_test, pred_test)

# mae_val = mean_absolute_error(y_val, preds)
# mse_val = mean_squared_error(y_val, preds)
# r2_val = r2_score(y_val, preds)

# mlflow.log_metric("mae_test", mae_test)
# mlflow.log_metric("mse_test", mse_test)
# mlflow.log_metric("r2_test", r2_test)
# mlflow.log_metric("mae_train", mae_train)
# mlflow.log_metric("mse_train", mae_train)
# mlflow.log_metric("r2_train", mae_train)
# mlflow.log_metric("mae", mae_val)
# mlflow.log_metric("mse", mse_val)
# mlflow.log_metric("r2", r2_val)

# results = results.append(
#     {
#         "model": model_name,
#         "mae": mae_val,
#         "mse": mse_val,
#         "r2": r2_val,
#         "mae_test": mae_test,
#         "mse_test": mse_test,
#         "r2_test": r2_test,
#         "mae_train": mae_train,
#         "mse_train": mse_train,
#         "r2_train": r2_train,
#     },
#     ignore_index=True,
# )

# with open(f"models/{model_name}.pkl", "wb") as file:
#     pickle.dump(model, file)
# print(f"Model {model_name} saved locally as pickle file.")
# print("Done with train")
# mlflow.end_run()